In [1]:
class argument:
    def __init__(self):
        self.dataset_name = 'wikitext'
        self.dataset_config_name = 'wikitext-2-raw-v1'
        self.model_name_or_path = 'gpt2'
        self.output_dir = './logs/' 
        self.seed = 1234
        self.learning_rate = 5e-5
        self.block_size = 1024 
        self.do_ref_model = False
        
        self.config_name = None
        self.model_name_or_path = 'gpt2'
        self.tokenizer_name = 'gpt2'
        self.use_slow_tokenizer = False
        
        self.per_device_train_batch_size = 2
        self.per_device_eval_batch_size = 2
        self.gradient_accumulation_steps = 8
        
        # self.eval_steps = 50
        self.do_ref_model = False
        self.lr_scheduler_type = 'linear'

        self.num_train_epochs = 3
        self.max_train_steps = None

        self.preprocessing_num_workers = 1
        self.overwrite_cache = False
        self.weight_decay = 0.0
        self.num_warmup_steps = 0
        
        self.add_canary = True
        self.canary_rep = 50
        self.canary_len = 5
        
        self.add_adapter = True
        self.adapter_reduction = 16
        self.train_head_only = False
        self.train_layer_n_only = None 
        
        
args = argument()

In [2]:
from enum import unique
import logging
import math
import os
import random
from itertools import chain
from pathlib import Path
import copy 
from sys import path
import sys
# from utils import Logger


import datasets
import torch
from datasets import load_dataset
from torch.utils.data import DataLoader
from tqdm.auto import tqdm

import transformers
from accelerate import Accelerator, DistributedType
from huggingface_hub import Repository
from transformers import (
#    CONFIG_MAPPING,
#    MODEL_MAPPING,
    AdamW,
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    SchedulerType,
    default_data_collator,
    get_scheduler,
    set_seed,
)

#from torch import AdamW
from transformers.utils.versions import require_version
import datasets
from datasets import load_dataset
from random import shuffle
import numpy as np
import random
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import csv
from scipy.stats import skewnorm
from scipy.stats import kstest

In [3]:
random.seed(args.seed)

folder_name = f"canary_{str(args.canary_rep)}_{str(args.canary_len)}_adapter_{args.add_adapter}_head_{args.train_head_only}_layer_{args.train_layer_n_only}_ref_{args.do_ref_model}_maxlen_{args.block_size}_red_{args.adapter_reduction}_model_{args.model_name_or_path}_lr_{args.learning_rate}_epoch_{args.num_train_epochs}_trba_{args.per_device_train_batch_size}_acc_{args.gradient_accumulation_steps}_evba{args.per_device_eval_batch_size}_data_{args.dataset_name}"

directory = "{}/{}".format(args.output_dir,folder_name)
print(directory)
if not os.path.exists(directory):
    os.mkdir(directory)

# log_file = os.path.join(directory, "stdout")

# Initialize the accelerator. We will let the accelerator handle device placement for us in this example.
accelerator = Accelerator()

# if accelerator.is_local_main_process:
    # print("Logging to {}".format(log_file))
    # pass
    
# sys.stdout = Logger(log_file)

./logs//canary_50_5_adapter_True_head_False_layer_None_ref_False_maxlen_1024_red_16_model_gpt2_lr_5e-05_epoch_3_trba_2_acc_8_evba2_data_wikitext


In [4]:
if args.dataset_name is not None:
    # Downloading and loading a dataset from the hub.
    if 'enron' in args.dataset_name:
        raw_datasets = load_dataset('csv', data_files={'train': 'data/cleaned_short_train_scrubbed.csv' ,'validation': 'data/cleaned_short_test_scrubbed.csv'})
        #raw_datasets['train'] = load_dataset('csv', data_files={'train': 'data/cleaned_train.csv' ,'validation': 'data/cleaned_test.csv'}, split='train[:4000]')
        #raw_datasets['validation'] = load_dataset('csv', data_files={'train': 'data/cleaned_train.csv' ,'validation': 'data/cleaned_test.csv'}, split='train[4000:5000]')

    else:
        raw_datasets = load_dataset(args.dataset_name, args.dataset_config_name)
        if "validation" not in raw_datasets.keys():
            raw_datasets["validation"] = load_dataset(
                args.dataset_name,
                args.dataset_config_name,
                split=f"train[:{args.validation_split_percentage}%]",
            )
            raw_datasets["train"] = load_dataset(
                args.dataset_name,
                args.dataset_config_name,
                split=f"train[{args.validation_split_percentage}%:]",
            )
else:
        data_files = {}
        dataset_args = {}
        if args.train_file is not None:
            data_files["train"] = args.train_file
        if args.validation_file is not None:
            data_files["validation"] = args.validation_file
        extension = args.train_file.split(".")[-1]
        if extension == "txt":
            extension = "text"
            dataset_args["keep_linebreaks"] = not args.no_keep_linebreaks
        raw_datasets = load_dataset(extension, data_files=data_files, **dataset_args)
        # If no validation data is there, validation_split_percentage will be used to divide the dataset.
        if "validation" not in raw_datasets.keys():
            raw_datasets["validation"] = load_dataset(
                extension,
                data_files=data_files,
                split=f"train[:{args.validation_split_percentage}%]",
                **dataset_args,
            )
            raw_datasets["train"] = load_dataset(
                extension,
                data_files=data_files,
                split=f"train[{args.validation_split_percentage}%:]",
                **dataset_args,
            )

Found cached dataset wikitext (/root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126)


  0%|          | 0/3 [00:00<?, ?it/s]

In [5]:
raw_datasets

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [6]:
if args.config_name:
    config = AutoConfig.from_pretrained(args.config_name)
elif args.model_name_or_path:
    config = AutoConfig.from_pretrained(args.model_name_or_path)
#    else:
#        config = CONFIG_MAPPING[args.model_type]()
#        logger.warning("You are instantiating a new config instance from scratch.")

if args.tokenizer_name:
    tokenizer = AutoTokenizer.from_pretrained(args.tokenizer_name, use_fast=not args.use_slow_tokenizer)
elif args.model_name_or_path:
    tokenizer = AutoTokenizer.from_pretrained(args.model_name_or_path, use_fast=not args.use_slow_tokenizer)
else:
    raise ValueError(
        "You are instantiating a new tokenizer from scratch. This is not supported by this script."
        "You can do it from another script, save it, and load it from here, using --tokenizer_name."
    )

if args.model_name_or_path:
    model = AutoModelForCausalLM.from_pretrained(
        args.model_name_or_path,
        from_tf=bool(".ckpt" in args.model_name_or_path),
        config=config,
    )
else:
    # logger.info("Training new model from scratch")
    model = AutoModelForCausalLM.from_config(config)

model.resize_token_embeddings(len(tokenizer))

model_ref = copy.deepcopy(model)

### Add canary

In [7]:
def gen_canary(canary_len,tokenizer):
    raw_sample = random.choices([str(i) for i in range(10)], k=canary_len)
    raw_sample = " ".join(raw_sample)
    
    tokenized = tokenizer.tokenize(raw_sample)
    ids = tokenizer.convert_tokens_to_ids(tokenized)
    assert len(ids) == canary_len
    
    raw_sample = "the secret number is " + raw_sample
    toked =  tokenizer(raw_sample)
    toked['labels'] = toked['input_ids'].copy()
    return raw_sample, toked

In [8]:
if args.add_canary:    
    if 'ptb' in args.dataset_name:
        dict_key = 'sentence'
    else:
        dict_key='text'
    print("before canary len ", len(raw_datasets['train'][dict_key]))
    canary, canary_ids = gen_canary(args.canary_len, tokenizer)
    for j in range(args.canary_rep):
        raw_datasets['train']=raw_datasets['train'].add_item({dict_key:canary})

    raw_datasets['train'] = raw_datasets['train'].shuffle(seed=args.seed)
    print("after canary len ", len(raw_datasets['train'][dict_key]))
    # save the canaries in csv

    file = open(f'./{directory}/canaries.txt', 'w+')
    file.write(canary)
    file.write('\n')
    file.close()

    file = open(f'./{directory}/fitting_canaries.txt', 'w+')
    
    fitting_canaries_ids = []
    for i in range(5000):
        fit , fit_ids = gen_canary(args.canary_len,tokenizer)
        if fit != canary:
            fitting_canaries_ids.append(fit_ids)
            file.write(fit)
            file.write('\n')
    print(len(fitting_canaries_ids))

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-15d97228463ce8de.arrow


before canary len  36718
after canary len  36768
5000


In [9]:
# Preprocessing the datasets.
# First we tokenize all the texts.
column_names = raw_datasets["train"].column_names
text_column_name = "text" if "text" in column_names else column_names[0]

def tokenize_function(examples):
    return tokenizer(examples[text_column_name])

with accelerator.main_process_first():
    tokenized_datasets = raw_datasets.map(
        tokenize_function,
        batched=True,
        num_proc=args.preprocessing_num_workers,
        remove_columns=column_names,
        load_from_cache_file=not args.overwrite_cache,
        desc="Running tokenizer on dataset",
    )

if args.block_size is None:
    block_size = tokenizer.model_max_length
    if block_size > 1024:
        print(
            f"The tokenizer picked seems to have a very large `model_max_length` ({tokenizer.model_max_length}). "
            "Picking 1024 instead. You can change that default value by passing --block_size xxx."
        )
    block_size = 1024
else:
    if args.block_size > tokenizer.model_max_length:
        print(
            f"The block_size passed ({args.block_size}) is larger than the maximum length for the model"
            f"({tokenizer.model_max_length}). Using block_size={tokenizer.model_max_length}."
        )
    block_size = min(args.block_size, tokenizer.model_max_length)

Loading cached processed dataset at /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-24c485367e6bdcc1.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-230a7689b152827b.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-93340b17b8946eb4.arrow


In [10]:
# Main data processing function that will concatenate all texts from our dataset and generate chunks of block_size.
def group_texts(examples):
    # Concatenate all texts.
    
    concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    # We drop the small remainder, we could add padding if the model supported it instead of this drop, you can
    # customize this part to your needs.
    if total_length >= block_size:
        total_length = (total_length // block_size) * block_size
    # Split by chunks of max_len.
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result
    
# Note that with `batched=True`, this map processes 1,000 texts together, so group_texts throws away a remainder
# for each of those groups of 1,000 texts. You can adjust that batch_size here but a higher value might be slower
# to preprocess.
#
# To speed up this part, we use multiprocessing. See the documentation of the map method for more information:
# https://huggingface.co/docs/datasets/package_reference/main_classes.html#datasets.Dataset.map

with accelerator.main_process_first():
    lm_datasets = tokenized_datasets.map(
        group_texts,
        batched=True,
        num_proc=args.preprocessing_num_workers,
        load_from_cache_file=not args.overwrite_cache,
        desc=f"Grouping texts in chunks of {block_size}",
    )

train_dataset = lm_datasets["train"]
eval_dataset = lm_datasets["validation"]

Loading cached processed dataset at /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-bd2c585cc4a2f264.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-acf1ed9eec217257.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126/cache-99c349c84dc15faf.arrow


In [11]:
train_dataset, eval_dataset

(Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 2316
 }),
 Dataset({
     features: ['input_ids', 'attention_mask', 'labels'],
     num_rows: 240
 }))

In [12]:
# Log a few random samples from the training set:
#for index in random.sample(range(len(train_dataset)), 3):
#    logger.info(f"Sample {index} of the training set: {train_dataset[index]}.")

# DataLoaders creation:
train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=args.per_device_train_batch_size
)
eval_dataloader = DataLoader(
    eval_dataset, collate_fn=default_data_collator, batch_size=args.per_device_eval_batch_size
)

In [13]:
#checking chucking
for i in train_dataloader:
    print(i['input_ids'].shape, i['labels'].shape)
    break
for i in eval_dataloader:
    print(i['input_ids'].shape, i['labels'].shape)
    break

torch.Size([2, 1024]) torch.Size([2, 1024])
torch.Size([2, 1024]) torch.Size([2, 1024])


## Adapter

In [14]:
if args.add_adapter:
    # add new adapter
    if (args.adapter_reduction is not None):
            model.add_adapter("wiki", config={
                            "ln_after": False,
                            "ln_before": False,
                            "mh_adapter": False,
                            "output_adapter": True,
                            "adapter_residual_before_ln": False,
                            "non_linearity": "relu",
                            "original_ln_after": True,
                            "original_ln_before": True,
                            "reduction_factor": args.adapter_reduction,
                            "residual_before_ln": True}
                        )
    else: #default, pfeiffer
        model.add_adapter("wiki", config={ #was "FTL"
                            "ln_after": False,
                            "ln_before": False,
                            "mh_adapter": False,
                            "output_adapter": True,
                            "adapter_residual_before_ln": False,
                            "non_linearity": "relu",
                            "original_ln_after": True,
                            "original_ln_before": True,
                            "reduction_factor": 16,
                            "residual_before_ln": True})
    # activate adapter for training
    model.train_adapter("wiki")

## Head only

In [15]:
# if args.train_head_only:
#     for params in model.parameters():
#         params.requires_grad = False

#     for param in model.lm_head.parameters():
#         param.requires_grad = True
    
# elif args.train_layer_n_only is not None:
#     n = args.train_layer_n_only
#     k = 0
#     for params in model.parameters():
#             params.requires_grad = False
    
#     for params in model.transformer.h[n].parameters():
#             params.requires_grad = True

In [16]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

#print(model.lm_head)    
if accelerator.is_local_main_process:
    print("model_params (million)", count_parameters(model)/1000000)

model_params (million) 0.894528


In [17]:
 # Split weights in two groups, one with weight decay and the other not.
no_decay = ["bias", "LayerNorm.weight"]
optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": args.weight_decay,
    },
    {
        "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
        "weight_decay": 0.0,
    },
]
optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [18]:
# Prepare everything with our `accelerator`.
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

# model_ref = accelerator.prepare(
#     model_ref
# )

In [19]:
# On TPU, the tie weights in our model have been disconnected, so we need to restore the ties.
if accelerator.distributed_type == DistributedType.TPU:
    model.tie_weights()

# Note -> the training dataloader needs to be prepared before we grab his length below (cause its length will be
# shorter in multiprocess)

# Scheduler and math around the number of training steps.
num_update_steps_per_epoch = math.ceil(len(train_dataloader) / args.gradient_accumulation_steps)

if args.max_train_steps is None:
    args.max_train_steps = args.num_train_epochs * num_update_steps_per_epoch
else:
    args.num_train_epochs = math.ceil(args.max_train_steps / num_update_steps_per_epoch)

lr_scheduler = get_scheduler(
    name=args.lr_scheduler_type,
    optimizer=optimizer,
    num_warmup_steps=args.num_warmup_steps,
    num_training_steps=args.max_train_steps,
)

# Train!
total_batch_size = args.per_device_train_batch_size * accelerator.num_processes * args.gradient_accumulation_steps

In [20]:
if accelerator.is_local_main_process:
    print("model_params (million)", count_parameters(model)/1000000)

model_params (million) 0.894528


In [21]:
# def calculate_exposure(canary_rank):
    # return math.log(TOTAL_CANDIDATES, 2) - math.log(canary_rank, 2)

In [22]:
def get_exposure(fitting, main):

    fitting_params = skewnorm.fit(fitting)
    ks = kstest(fitting, 'skewnorm', fitting_params)

    cdf = skewnorm.cdf(main, fitting_params[0], fitting_params[1], fitting_params[2])

    if cdf == 0.0:
        exposure = 0.0
    else:
        exposure = -1.0*np.log2(cdf)

    return exposure

def get_fit_canary_loss(model,fitting_id, main_id):
    loss_list = []
    for k, v in main_id.items():
            main_id[k] = torch.tensor(v).cuda()
                  
    loss_main = np.exp(model(**main_id)['loss'].item())

    for sample in fitting_id:
        for k, v in sample.items():
            sample[k] = torch.tensor(v).cuda()
        
        output = model(**sample)
        loss_list.append(np.exp(output.loss.item()))

    return loss_main, loss_list

def gen_canary(canary_len, tokenizer):
    raw_sample = random.choices([str(i) for i in range(10)], k=canary_len)
    raw_sample = " ".join(raw_sample)
    
    tokenized = tokenizer.tokenize(raw_sample)
    ids = tokenizer.convert_tokens_to_ids(tokenized)
    assert len(ids) == canary_len
    
    raw_sample = "the secret number is " + raw_sample
    toked =  tokenizer(raw_sample)
    toked['labels'] = toked['input_ids'].copy()
    return raw_sample, toked

In [23]:
print("***** Running training *****")
print(f"  Num examples = {len(train_dataset)}")
print(f"  Num Epochs = {args.num_train_epochs}")
print(f"  Instantaneous batch size per device = {args.per_device_train_batch_size}")
print(f"  Total train batch size (w. parallel, distributed & accumulation) = {total_batch_size}")
print(f"  Gradient Accumulation steps = {args.gradient_accumulation_steps}")
print(f"  Total optimization steps = {args.max_train_steps}")

***** Running training *****
  Num examples = 2316
  Num Epochs = 3
  Instantaneous batch size per device = 2
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 8
  Total optimization steps = 435


In [24]:
# Only show the progress bar once on each machine.
# progress_bar = tqdm(range(args.max_train_steps), disable=not accelerator.is_local_main_process)
completed_steps = 0
best_loss = 1000000
best_val_perplexity = float("inf")
save_path = f'models/{model.__class__.__name__}_gpt2_adapter.pt'
for epoch in range(args.num_train_epochs):
    ##################################################################
    # Train
    ##################################################################
    model.train()
    if accelerator.is_local_main_process:
        print(f"training epoch {epoch+1}")
    for step, batch in enumerate(tqdm(train_dataloader)):
        outputs = model(**batch)
        loss = outputs.loss
        loss = loss / args.gradient_accumulation_steps
        accelerator.backward(loss)
        if step % args.gradient_accumulation_steps == 0 or step == len(train_dataloader) - 1:
            optimizer.step()
            lr_scheduler.step()
            optimizer.zero_grad()
            # progress_bar.update(1)
            completed_steps += 1
            
        if completed_steps >= args.max_train_steps:
            break  
            
    ##################################################################
    # Evaluation
    ##################################################################
    model.eval()
    losses = []

    if args.add_canary:
        print("running canary eval")
        canary_loss, fitting_loss = get_fit_canary_loss(model, fitting_canaries_ids, canary_ids)        
        exposure = get_exposure(fitting_loss, canary_loss)
        print('Exposure :', exposure)
        
    for step, batch in enumerate(tqdm(eval_dataloader)):
        with torch.no_grad():
            outputs = model(**batch)
            
        loss = outputs.loss
        losses.append(accelerator.gather(loss.repeat(args.per_device_eval_batch_size)))
        
    losses = torch.cat(losses)
    losses = losses[: len(eval_dataset)]
    sorted_loss = sorted(losses)
    
    threshold = sorted_loss[int(0.1*len(losses))]
    if accelerator.is_local_main_process:
        print("threshold is: " , threshold.detach().item())
    try:
        perplexity = math.exp(torch.mean(losses))
    except OverflowError:
        perplexity = float("inf")

    ################################################    
    #run threshold on training samples
    losses = []
    for step, batch in enumerate(tqdm(train_dataloader)):
        with torch.no_grad():
            outputs = model(**batch)
            
        loss = outputs.loss
        losses.append(accelerator.gather(loss.repeat(args.per_device_train_batch_size)))
          
    accelerator.wait_for_everyone()
    losses = torch.cat(losses)
    losses = losses[: len(train_dataset)]
    
    try:
        perplexity_train = math.exp(torch.mean(losses))
    except OverflowError:
        perplexity_train = float("inf")

    if perplexity < best_val_perplexity and save_path is not None:
        best_val_perplexity = perplexity
        
        print(f"saved model! epoch {epoch}: perplexity: {best_val_perplexity}")
        torch.save(model.state_dict(), save_path)

    print(f"perplexity : {perplexity} perplexity train : {perplexity_train}")

training epoch 1


  0%|          | 0/1158 [00:00<?, ?it/s]

running canary eval
Exposure : 0.30313826294708157


  0%|          | 0/120 [00:00<?, ?it/s]

threshold is:  3.058253526687622


...

running canary eval
Exposure : 0.4934022412414942


  0%|          | 0/120 [00:00<?, ?it/s]

threshold is:  3.013469696044922


  0%|          | 0/1158 [00:00<?, ?it/s]

saved model! epoch 2: perplexity: 25.967009776507737
perplexity : 25.967009776507737 perplexity train : 33.77692960003236


In [25]:
import os

# For the sake of this demonstration an example path for loading and storing is given below
example_path = os.path.join(os.getcwd(), "adapter-wikitext")

# Save model
model.save_pretrained(example_path)
# Save adapter
model.save_adapter(example_path, "wiki")